This notebook runs a `Qallse` model in simulation mode and shows how to simply visualize the results using utilities defined in module `hepqpr.qallse.plotting`.

# Import and setup

In [1]:
# import modules
import pandas as pd
import numpy as np

from hepqpr.qallse.plotting import *
from hepqpr.qallse import *
from hepqpr.qallse.dsmaker import create_dataset

# initialise the plotting module in "notebook" mode
set_notebook_mode()

# initialise the logging module
import logging
logging.basicConfig()
fmt = logging.Formatter("%(asctime)s.%(msecs)03d %(levelname)s %(name)s: %(message)s", datefmt='%H:%M:%S')
for handler in logging.getLogger().handlers: handler.setFormatter(fmt)
logging.getLogger('hepqpr').setLevel(logging.DEBUG)

0
1
2
3
[(-4, -1), (-2, 1), (0, 3), (2, 5), (4, inf), (-inf, -3)]
[(0.0, 0.7)]
[(0.0, 0.7), (0.5, 1.2)]
[(0.0, 0.7), (0.5, 1.2), (1.0, 1.7)]
[(0.0, 0.7), (0.5, 1.2), (1.0, 1.7), (1.5, 2.2)]


# Define run config

In [2]:
# == DATASET CONFIG

dsmaker_config = dict(
    density=.10, # 1% 
)

# == INPUT CONFIG

# whether or not to add missing doublets to the input
add_missing = True 

# == RUN CONFIG

model_class = Qallse # model class to use
extra_config = dict() # configuration arguments overriding the defaults

# Generate the data

In [3]:
import tempfile

tempdir = tempfile.TemporaryDirectory()
print(f'using {tempdir.name}')
#random_seed=240834366
metas, path = create_dataset(output_path=tempdir.name, gen_doublets=True, random_seed=240834351, **dsmaker_config)

using /var/folders/7r/y3xzyywn4mg2tfhg0sgjqx_w0000gn/T/tmpk5zp0q9u


17:20:28.488 DEBUG hepqpr.qallse.dsmaker.dsmaker: Loaded 120939 hits from /Users/parkerreid/hepqpr-qallse/src/hepqpr/qallse/dsmaker/data/event000001000.
17:20:28.511 DEBUG hepqpr.qallse.dsmaker.dsmaker: Filtered hits from barrel. Remaining hits: 65518.
17:20:28.559 DEBUG hepqpr.qallse.dsmaker.dsmaker: Dropped double hits. Remaining hits: 55446.
17:20:28.598 DEBUG hepqpr.qallse.dsmaker.dsmaker: High Pt hits: 32302/5484
17:20:28.808 DEBUG hepqpr.qallse.dsmaker.dsmaker:   num_hits=5484
17:20:28.809 DEBUG hepqpr.qallse.dsmaker.dsmaker:   num_tracks=818
17:20:28.809 DEBUG hepqpr.qallse.dsmaker.dsmaker:   num_important_tracks=60
17:20:28.810 DEBUG hepqpr.qallse.dsmaker.dsmaker:   num_noise=1303
17:20:28.811 DEBUG hepqpr.qallse.dsmaker.dsmaker:   random_seed=240834351
17:20:28.811 DEBUG hepqpr.qallse.dsmaker.dsmaker:   time=2020-01-06T17:20:28.808193
17:20:31.709 INFO hepqpr.qallse.dsmaker.dsmaker: Doublets (len=100811) generated in f/var/folders/7r/y3xzyywn4mg2tfhg0sgjqx_w0000gn/T/tmpk5zp0q9

In [4]:
with open(path + '-meta.json') as f:
    print(f.read())

{
    "num_hits": 5484,
    "num_tracks": 818,
    "num_important_tracks": 60,
    "num_noise": 1303,
    "random_seed": 240834351,
    "time": "2020-01-06T17:20:28.808193",
    "params": {
        "input_path": "/Users/parkerreid/hepqpr-qallse/src/hepqpr/qallse/dsmaker/data/event000001000",
        "output_path": "/var/folders/7r/y3xzyywn4mg2tfhg0sgjqx_w0000gn/T/tmpk5zp0q9u",
        "density": 0.1,
        "high_pt_cut": 1.0,
        "double_hits_ok": false,
        "gen_doublets": true,
        "prefix": null,
        "random_seed": 240834351,
        "phi_bounds": null,
        "min_hits_per_track": 5
    }
}


# Execute the model

## Load the data

In [5]:
# load data
dw = DataWrapper.from_path(path)
doublets = pd.read_csv(path + '-doublets.csv')
if add_missing:
    doublets = dw.add_missing_doublets(doublets)
else:
    p, r, ms = dw.compute_score(doublets)
    print(f'got {len(doublets)}.')
    print(f'  Input precision (%): {p*100:.4f}, recall (%): {r*100:.4f}, missing: {len(ms)}')

got 100811 doublets.
  Input precision (%): 0.4427, recall (%): 98.1982
    New precision (%): 0.4508


## Build the model

In [6]:
%%time

# instantiate qallse
model = model_class(dw, **extra_config)
# build the qubo
model.build_model(doublets=doublets)
sliceContainer = model.to_qubo()

17:20:33.176 DEBUG hepqpr.qallse.qallse: using config:
17:20:33.177 DEBUG hepqpr.qallse.qallse:     cheat: False
17:20:33.178 DEBUG hepqpr.qallse.qallse:     max_layer_span: 2
17:20:33.179 DEBUG hepqpr.qallse.qallse:     num_multiplier: -1
17:20:33.179 DEBUG hepqpr.qallse.qallse:     qplet_max_dcurv: 0.0001
17:20:33.180 DEBUG hepqpr.qallse.qallse:     qplet_max_strength: -0.2
17:20:33.181 DEBUG hepqpr.qallse.qallse:     qubo_bias_weight: 0
17:20:33.181 DEBUG hepqpr.qallse.qallse:     qubo_conflict_strength: 1
17:20:33.182 DEBUG hepqpr.qallse.qallse:     rz_power: 1
17:20:33.182 DEBUG hepqpr.qallse.qallse:     strength_bounds: None
17:20:33.183 DEBUG hepqpr.qallse.qallse:     tplet_max_curv: 0.0008
17:20:33.183 DEBUG hepqpr.qallse.qallse:     tplet_max_drz: 0.1
17:20:33.184 DEBUG hepqpr.qallse.qallse:     volayer_power: 2
17:20:33.184 DEBUG hepqpr.qallse.qallse:     xy_power: 1
17:20:33.187 DEBUG hepqpr.qallse.qallse:     xy_relative_strength: 0.5


-15.6292 28.1946 -3.174685354812164
-32.26 -11.821 -2.831755383640396
-29.7923 -14.5837 -2.838233157201391
4.19956 -32.3618 -2.9658998211918584
23.1078 -22.1786 -2.9798917273286274
27.3535 -16.7994 -2.928768492241476
31.4773 6.72935 -2.9828696496310525
28.9097 14.0729 -2.9102258620484034
22.8218 22.3877 -2.9155732706129025
18.1284 26.7131 -2.9096069106187787
4.32987 31.741 -2.9100520969143906
9.39842 31.4551 -2.939621006381612
-17.5603 27.1336 -2.8554529869755036
-20.0788 24.8841 -2.863576138525573
-27.5686 16.352999999999998 -2.9933542367419057
-30.9966 -6.6042 -2.8174431722918394
-23.4727 -20.9158 -2.590990861602686
-26.4661 -17.5644 -2.781605431302997
-28.5473 -15.2344 -2.692689015416243
-19.3085 -25.5948 -2.78416513959452
-15.0567 -27.6437 -2.6723456767618905
-10.1621 -30.9161 -2.6472462731160196
19.4975 -24.7394 -2.7114987143070715
18.2881 -25.8197 -2.7128401890245857
31.374 -2.04447 -2.6299934505541205
31.5028 0.238742 -2.6755187212418092
28.6999 13.1729 -2.7446884495630877
31.97

2.00999 31.376 0.6890192655029262
1.5419200000000002 33.8447 0.6026590402747787
-8.717239999999999 30.2578 -0.37686381946952857
-13.6956 28.5172 -2.53178211433266
-6.80139 30.9276 0.9446424535072244
-13.0785 28.7329 0.6153879893144362
-8.73738 30.2507 0.957897358984799
-10.7082 29.5616 0.11172499567900783
-12.5122 28.9309 0.7035161818802554
-11.6849 29.2202 0.3598938614217349
-5.10187 31.5218 -1.0942196321650859
-18.5149 25.6171 1.4167610871140908
-19.7282 24.5334 1.3121695769544384
-14.7556 28.975 0.2716991392918013
-15.3387 28.4542 0.05995173150422083
-14.8426 28.8973 1.1736840993141753
-21.8553 22.6335 0.6170082088575615
-20.9652 23.4285 1.1183038573031208
-28.3931 13.5016 1.3505464850553643
-25.5168 19.4704 0.8722977414729663
-25.9992 18.4693 0.3774153378654271
-27.1791 16.0209 1.2068451699580898
-25.6118 19.2734 1.195816921069383
-28.5884 13.0962 -0.32594636929709914
-29.2608 11.7009 0.39187713880656094
-25.164 20.2026 -0.37686059457631926
-27.2121 15.9524 0.5770797369513088
-30.0

12.6684 71.6404 0.5692041363631817
11.3516 71.5852 0.5408311925105697
12.9032 71.6502 0.41317349585899193
2.48 71.214 0.5964676478300289
-4.93588 71.9136 -0.5180162003688744
-13.0885 69.9343 -0.8736295324823297
-28.0782 65.2955 0.348367317737315
-25.7165 66.3917 0.1586949921214308
-24.2248 67.0839 -2.5901042078504815
-15.9383 70.9298 -0.46307156633636826
-31.6931 64.8335 0.18482194111395941
-31.0547 65.2997 -0.29728700428448884
-37.6528 60.4801 -0.07631865538757979
-44.0972 57.0081 -1.6092864966948601
-47.2022 53.6318 0.6179476253051377
-56.3161 44.1183 -3.5802341361570384
-59.0026 39.7091 0.07209708570526346
-56.1014 44.4708 -0.18541225218032362
-63.2108 34.006 0.5819029043261468
-65.9527 26.5148 0.3924237173270594
-67.9263 25.5412 -0.13953899874261333
-71.262 3.62811 0.5822814375738034
-71.2181 2.57783 0.6348532014920867
-71.9358 -6.94305 1.1928766747910247
-67.2648 -25.0111 0.752733591250338
-67.6393 -24.2043 0.8532929505806314
-67.1687 -29.3355 1.2465391331194835
-65.7417 -31.2891 

79.1696 84.4838 0.7063823854671416
66.1084 94.7419 0.7519550693212124
72.6198 90.9741 0.33061347364936594
74.8356 89.69200000000002 0.5454327776456558
64.5034 97.6207 0.3812289250659713
30.8524 111.597 0.7214914982482812
21.9198 114.306 0.49227113856039423
21.2305 114.334 0.6348594865695205
25.3135 114.166 0.37243455318441504
20.5087 114.364 0.5206905024970343
2.35462 115.666 0.5947163599489529
-0.0516349 115.474 0.7287226435466249
-29.0853 113.589 0.7714187889866879
-36.5076 110.076 0.7310072590589766
-48.6781 105.398 0.34406090331259154
-55.922 102.922 0.7342080853307447
-71.9974 91.1944 0.6238618453675988
-104.869 51.6283 0.5845789727442376
-110.302 38.7705 0.3985432986772346
-116.101 7.81302 0.6130249052623582
-116.186 8.8762 0.6753840854740263
-116.282 10.0851 0.7514109922163806
-114.64 -11.415 1.1754169044975238
-114.64 -11.4134 1.175445074531177
-107.061 -45.1147 1.2341261516294366
-103.086 -51.4145 1.1395780088558811
-101.076 -57.4035 1.202455276547755
-81.813 -81.8737 0.982633

...

-407.545 -518.775 1.5501615333688452
627.2090000000002 -193.723 -0.7646482615884551
-256.839 -991.228 -0.9369751365688914
-842.041 -579.155 0.4972849406732197
446.675 -915.267 0.22505612468339953
67.7472 96.235 1.965917156416369
213.987 -147.297 -0.4353635683236124
95.4245 144.067 -1.6264576825742514
-23.0088 -658.023 0.8535753873244757
433.523 696.832 -0.03607806477008827
988.411 -239.356 1.5456171121681117
263.38 250.315 -1.5550489085073629
448.369 -221.98 -0.29525893035454015
580.758 -311.087 0.3618488538336302
-656.547 82.3931 0.3880402770862049
-215.378 -1001.11 0.704202784587191
-773.699 -658.2769999999998 0.5884206853899211
-1002.08 212.924 -3.4537542418617493
-17.6111 360.681 -0.3246232304218204
-16.1423 -258.666 0.7161503147435065
818.097 -607.247 0.0003548822806832689
-436.001 924.907 -0.26531061706220227
-205.55 -455.83 0.4410260403235231
46.2966 -814.694 1.0639370096796228
-495.521 -82.5678 -1.3019066973252365
-183.88 -465.714 1.3601909441739934
-171.24200000000005 -797.15 

661.934 26.6162 -0.27737349252125193
-204.57 293.833 -0.9219816034772056
693.426 445.843 0.5704556760927391
-250.016 -987.377 2.349353644391801
-972.052 -297.668 -1.9003461131847188
522.594 632.756 1.331134967848519
-697.682 432.513 0.9148399159769309
-1019.55 91.8693 0.34250860177041076
114.419 14.9894 1.9224542592884646
377.03 544.885 -0.595740604061507
-503.645 -422.707 0.8479955877384893
857.242 554.923 0.6000607878873463
-59.4593 -161.977 -1.0764050806830048
-272.366 -231.161 1.448064730450357
-282.958 -976.358 -0.01880110598201127
-1010.88 -156.946 0.26556679521563675
-88.8968 148.27100000000004 1.4919069351530423
600.682 -282.701 0.7693035165605863
628.4159999999998 805.612 -0.27475513676837615
-751.175 327.295 0.4757088127735846
-367.9980000000001 -728.189 -1.6804029686757562
-5.97586 -31.1522 3.085616544826316
350.494 358.092 -0.1986689076415888
842.8739999999998 570.599 -0.10204355442894888
-321.625 971.189 0.9360928579269592
7.54838 -71.7888 2.3825544785543884
432.776 246.54

727.183 716.107 -1.3674547327822404
-234.206 991.931 0.100713419957342
163.174 1002.9 -0.6277111825594527
917.461 -438.3 -0.5893418025597874
-561.817 857.493 2.607137969610795
140.899 -99.5675 -0.12927828710610717
-438.505 231.308 0.6362463914969249
75.9495 86.9084 1.9819104076302976
497.601 52.1425 0.008110738335986222
-736.334 372.961 -0.34065009497060167
1022.61 31.3172 -0.5215882255276129
587.266 -572.202 0.8903542285842376
37.2247 -353.853 0.16793891789756107
286.273 -769.0369999999998 0.1681297942636754
156.683 -323.131 1.2971899914748939
877.885 -517.679 -0.8798282920131458
366.912 554.618 -0.41392243056085937
1003.22 207.301 -0.4284823586738562
-224.371 -450.435 1.1279645969960477
-374.942 944.641 -0.5513171194116274
640.783 -510.644 0.6616312484726032
-9.85456 -171.453 0.3393618280009174
-804.105 -169.88 -0.29658063888657044
378.99 542.999 -0.6537524342710613
-19.6224 114.789 1.3165930250174909
-814.827 -48.1518 -1.6808383385954266
890.67 498.064 0.5115398718925566
-498.495 58

17:20:36.652 INFO hepqpr.qallse.qallse: created 50809 doublets.
17:20:43.888 INFO hepqpr.qallse.qallse: created 10565 triplets.
17:20:44.014 INFO hepqpr.qallse.qallse: created 1341 quadruplets.
17:20:44.015 INFO hepqpr.qallse.qallse: Model built in 8.76s. doublets: 50809/1281, triplets: 10565/1416, quadruplets: 1341
17:20:44.019 INFO hepqpr.qallse.qallse: Dropped 5 real structures during preprocessing
17:20:44.022 DEBUG hepqpr.qallse.qallse: 
    type                      hid    reason      arg1  arg2
0  tplet        19988_28861_36080      curv -0.008034   NaN
1  tplet        28861_36080_42582      curv -0.003800   NaN
2  tplet        36080_42582_74065      curv  0.002523   NaN
3  tplet       88679_95572_112552      curv  0.002149   NaN
4  qplet  18978_27522_34094_71942  strength -0.183443   NaN
17:20:44.024 INFO hepqpr.qallse.qallse: Dropped type:reason:count => qplet:strength:1 tplet:curv:4 
17:20:44.062 INFO hepqpr.qallse.qallse: Qubo generated in 0.04s. Size: 409. Vars: 83, excl. c

CPU times: user 11.6 s, sys: 461 ms, total: 12.1 s
Wall time: 11.9 s


## Sample the QUBO

Here, we use [_qbsolv_](https://github.com/dwavesystems/qbsolv) with default arguments. This means only classical algorithms.

In [7]:
%%time
# execute the qubo
responseContainer = model.sample_qubo_slices(Q=sliceContainer)
#response = model.sample_qubo_slices(Q=qslice)

17:20:45.270 INFO hepqpr.qallse.qallse: QUBO of size 409 sampled in 0.23s (seed 523592239).


0   0
{('40666_72709_79271', '40666_72709_79271'): 0, ('40680_72723_79280', '40680_72723_79280'): 0, ('40680_79280_85652', '40680_79280_85652'): 0, ('19584_34696_40629', '19584_34696_40629'): 0, ('33899_40017_71831', '33899_40017_71831'): 0, ('33899_40017_78392', '33899_40017_78392'): 0, ('33899_71831_78392', '33899_71831_78392'): 0, ('36639_43011_74449', '36639_43011_74449'): 0, ('36639_74449_81454', '36639_74449_81454'): 0, ('36639_43011_81454', '36639_43011_81454'): 0, ('72347_79239_85398', '72347_79239_85398'): 0, ('72336_79238_85622', '72336_79238_85622'): 0, ('36803_43176_74582', '36803_43176_74582'): 0, ('36803_43176_82092', '36803_43176_82092'): 0, ('36803_74582_82092', '36803_74582_82092'): 0, ('72670_79233_85622', '72670_79233_85622'): 0, ('72723_79280_85652', '72723_79280_85652'): 0, ('27760_34681_72670', '27760_34681_72670'): 0, ('27760_40624_72670', '27760_40624_72670'): 0, ('27760_34681_40624', '27760_34681_40624'): 0, ('28666_43176_74582', '28666_43176_74582'): 0, ('1957

0   1
{('18679_33971_40066', '18679_33971_40066'): 0, ('18679_27311_33971', '18679_27311_33971'): 0, ('18679_27311_40066', '18679_27311_40066'): 0, ('18718_27370_34420', '18718_27370_34420'): 0, ('18718_27370_40407', '18718_27370_40407'): 0, ('18718_34420_40407', '18718_34420_40407'): 0, ('18746_26845_33678', '18746_26845_33678'): 0, ('40666_72709_79271', '40666_72709_79271'): 0, ('40680_72723_79280', '40680_72723_79280'): 0, ('40680_79280_85652', '40680_79280_85652'): 0, ('33899_40017_71831', '33899_40017_71831'): 0, ('33899_40017_78392', '33899_40017_78392'): 0, ('33899_71831_78392', '33899_71831_78392'): 0, ('33971_72121_78614', '33971_72121_78614'): 0, ('33971_40066_72121', '33971_40066_72121'): 0, ('72723_79280_85652', '72723_79280_85652'): 0, ('74738_81689_88324', '74738_81689_88324'): 0, ('88324_94647_110567', '88324_94647_110567'): 0, ('34420_72146_78839', '34420_72146_78839'): 0, ('34420_40407_78839', '34420_40407_78839'): 0, ('30135_37005_74738', '30135_37005_74738'): 0, ('30

17:20:45.489 INFO hepqpr.qallse.qallse: QUBO of size 309 sampled in 0.45s (seed 523592239).


0   2
{('27403_34451_72181', '27403_34451_72181'): 0, ('27403_40442_72181', '27403_40442_72181'): 0, ('27442_34473_72188', '27442_34473_72188'): 0, ('18718_27370_34420', '18718_27370_34420'): 0, ('27476_40465_72191', '27476_40465_72191'): 0, ('18718_27370_40407', '18718_27370_40407'): 0, ('27442_40456_72188', '27442_40456_72188'): 0, ('18718_34420_40407', '18718_34420_40407'): 0, ('27442_34473_40456', '27442_34473_40456'): 0, ('27486_40144_72205', '27486_40144_72205'): 0, ('27476_34483_72191', '27476_34483_72191'): 0, ('27476_34483_40465', '27476_34483_40465'): 0, ('18746_26845_33678', '18746_26845_33678'): 0, ('27486_34070_40144', '27486_34070_40144'): 0, ('18770_27403_40442', '18770_27403_40442'): 0, ('27495_40485_72555', '27495_40485_72555'): 0, ('27486_34070_72205', '27486_34070_72205'): 0, ('27495_34511_40487', '27495_34511_40487'): 0, ('18770_27403_34451', '18770_27403_34451'): 0, ('27495_40487_72209', '27495_40487_72209'): 0, ('18824_27442_40456', '18824_27442_40456'): 0, ('2749

...

2   2
{('74162_87490_93653', '74162_87490_93653'): 0, ('74249_81286_87956', '74249_81286_87956'): 0, ('22639_38355_45321', '22639_38355_45321'): 0, ('22654_30202_44453', '22654_30202_44453'): 0, ('22654_30202_37752', '22654_30202_37752'): 0, ('74249_87956_93719', '74249_87956_93719'): 0, ('74253_81287_87550', '74253_81287_87550'): 0, ('22654_37752_44453', '22654_37752_44453'): 0, ('74253_87550_93700', '74253_87550_93700'): 0, ('76802_83854_90958', '76802_83854_90958'): 0, ('76802_90958_97084', '76802_90958_97084'): 0, ('77133_84406_91378', '77133_84406_91378'): 0, ('43919_75849_89808', '43919_75849_89808'): 0, ('42195_74272_81317', '42195_74272_81317'): 0, ('43919_75849_83048', '43919_75849_83048'): 0, ('42195_74272_87581', '42195_74272_87581'): 0, ('38334_44897_76747', '38334_44897_76747'): 0, ('43919_83048_89808', '43919_83048_89808'): 0, ('42195_81317_87581', '42195_81317_87581'): 0, ('38334_44897_84106', '38334_44897_84106'): 0, ('38355_45321_77111', '38355_45321_77111'): 0, ('4220

17:20:51.214 INFO hepqpr.qallse.qallse: QUBO of size 1621 sampled in 6.17s (seed 523592239).


2   3
{('73903_87327_93201', '73903_87327_93201'): 0, ('73913_81013_87716', '73913_81013_87716'): 0, ('73913_81013_93850', '73913_81013_93850'): 0, ('73913_87716_93850', '73913_87716_93850'): 0, ('76522_83592_90553', '76522_83592_90553'): 0, ('76521_90553_96721', '76521_90553_96721'): 0, ('31499_38991_45756', '31499_38991_45756'): 0, ('31499_45756_77504', '31499_45756_77504'): 0, ('76579_83974_90832', '76579_83974_90832'): 0, ('76858_83906_90996', '76858_83906_90996'): 0, ('76858_90996_97117', '76858_90996_97117'): 0, ('76871_84197_91013', '76871_84197_91013'): 0, ('29485_36541_42944', '29485_36541_42944'): 0, ('29485_36541_75005', '29485_36541_75005'): 0, ('29485_42944_75005', '29485_42944_75005'): 0, ('23256_31338_38457', '23256_31338_38457'): 0, ('23256_31338_45023', '23256_31338_45023'): 0, ('23256_38457_45023', '23256_38457_45023'): 0, ('23265_31371_45032', '23265_31371_45032'): 0, ('38425_44988_76455', '38425_44988_76455'): 0, ('38425_44988_83887', '38425_44988_83887'): 0, ('3842

17:20:51.758 INFO hepqpr.qallse.qallse: QUBO of size 1272 sampled in 6.71s (seed 523592239).
17:20:51.771 INFO hepqpr.qallse.qallse: QUBO of size 54 sampled in 6.73s (seed 523592239).
17:20:51.785 INFO hepqpr.qallse.qallse: QUBO of size 54 sampled in 6.74s (seed 523592239).
17:20:51.807 INFO hepqpr.qallse.qallse: QUBO of size 36 sampled in 6.76s (seed 523592239).
17:20:51.821 INFO hepqpr.qallse.qallse: QUBO of size 97 sampled in 6.78s (seed 523592239).


3   0
{('74639_81591_88239', '74639_81591_88239'): 0, ('74639_81591_94227', '74639_81591_94227'): 0, ('74639_88239_94227', '74639_88239_94227'): 0, ('42547_74639_88239', '42547_74639_88239'): 0, ('42547_74639_81591', '42547_74639_81591'): 0, ('42547_81591_88239', '42547_81591_88239'): 0, ('36025_74639_81591', '36025_74639_81591'): 0, ('32516_39724_78205', '32516_39724_78205'): 0, ('32523_39731_78210', '32523_39731_78210'): 0, ('94227_110199_115361', '94227_110199_115361'): 0, ('81591_88239_94227', '81591_88239_94227'): 0, ('81591_94227_110199', '81591_94227_110199'): 0, ('23812_32516_39724', '23812_32516_39724'): 0, ('23859_32523_39731', '23859_32523_39731'): 0, ('88239_94227_110199', '88239_94227_110199'): 0, ('36025_74639_81591', '28775_36025_42547'): 1, ('42547_81591_88239', '42547_74639_88239'): 1, ('42547_81591_88239', '42547_74639_81591'): 1, ('36025_74639_81591', '42547_74639_88239'): 1, ('36025_74639_81591', '42547_74639_81591'): 1, ('74639_81591_94227', '74639_88239_94227'): 1

## Parse the results

In [8]:
# get all output doublets
all_doublets = model.process_sample_slices(responseContainer=responseContainer)
# recreate tracks and resolve remaining conflicts
final_tracks, final_doublets = TrackRecreaterD().process_results(all_doublets)

17:20:51.844 INFO hepqpr.qallse.track_recreater: Found 8 conflicting doublets
17:20:51.846 DEBUG hepqpr.qallse.track_recreater: Conflicts added: [21757, 38152], [18917, 34070], [30395, 37861].
17:20:51.847 INFO hepqpr.qallse.track_recreater: Added 3 conflicting doublets


# Evaluate the results

## Print statistics and scores

In [9]:
# stats about the qbsolv run
#en0 = dw.compute_energy(Q)
#en = response.record.energy[0]

#print(f'SAMPLE -- energy: {en:.4f}, ideal: {en0:.4f} (diff: {en-en0:.6f})')
#occs = response.record.num_occurrences
#print(f'          best sample occurrence: {occs[0]}/{occs.sum()}')

# scores
p, r, missings = dw.compute_score(final_doublets)
print(f'SCORE  -- precision (%): {p * 100}, recall (%): {r * 100}, missing: {len(missings)}')
trackml_score = dw.compute_trackml_score(final_tracks)
print(f'          tracks found: {len(final_tracks)}, trackml score (%): {trackml_score * 100}')

SCORE  -- precision (%): 96.83698296836984, recall (%): 89.63963963963964, missing: 46
          tracks found: 97, trackml score (%): 89.98633596339272


### Plot the results

You have the following at your disposal:
```python
iplot_results(dw, final_doublets, missings, dims=list('xy'))
iplot_results_tracks(dw, final_doublets, missings, dims=list('zr'))
iplot_results_any(dw, dw.real_doublets, dims=list('xy'))
```

For 3D plots, simply define `dims` as 3-dimensional. For example:
```python
iplot_results(dw, final_doublets, missings, dims=list('zxy'))
```

In [10]:
iplot_results(dw, final_doublets, missings, dims=list('xy'))

In [11]:
iplot_results(dw, final_doublets, missings, dims=list('zr'))

In [12]:
iplot_results(dw, final_doublets, missings, dims=list('zxy'))